In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
# 원할한 기사 크롤링을 위한 가이드 코드임
# 네이버에서 검색어를 "카카오"(큰 따옴표 포함)로 하고, 2020년 1월 1일 기사에 대해서 기사제목, 기사내용에 대해서 크롤링함
url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%22%EC%B9%B4%EC%B9%B4%EC%98%A4%22&sort=0&photo=0&field=0&pd=3&ds=2020.01.01&de=2020.01.01&cluster_rank=49&mynews=0&office_type=0&office_section_code=0&news_office_checked=&office_category=0&service_area=0&nso=so:r,p:from20200101to20200101,a:all&start=1'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
soup

<!DOCTYPE html>
 <html lang="ko"><head> <meta charset="utf-8"/> <meta content="always" name="referrer"/> <meta content="telephone=no,address=no,email=no" name="format-detection"/> <meta content='"카카오" : 네이버 뉴스검색' property="og:title"> <meta content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png" property="og:image"/> <meta content="'&quot;카카오&quot;'의 네이버 뉴스검색 결과입니다." property="og:description"/> <meta content="'&quot;카카오&quot;'의 네이버 뉴스검색 결과입니다." lang="ko" name="description"/> <title>"카카오" : 네이버 뉴스검색</title> <link href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_191118_pc.ico" rel="shortcut icon"/> <link href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" rel="search" title="Naver" type="application/opensearchdescription+xml"><script> if (top.frames.length!=0 || window!=top) window.open(location, "_top"); </script><link href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_231026.css" rel="stylesheet" type="text/css"/> <link href="htt

In [ ]:
# 기사 제목, 기사 내용을 크롤링함
# 기사 제목은 위 사이트에서 바로 진행 가능하나, 기사 내용은 '네이버 뉴스'링크로 들어가서 기사 내용을 크롤링함
# 네이버 뉴스 링크가 없는 기사는 기사 내용에 대해 None처리함
title = soup.select('div.news_contents > a.news_tit')
title

[<a class="news_tit" href="https://imnews.imbc.com/replay/2020/nw1200/article/5646598_32496.html" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=50&amp;i=88000119_000000000000000001004356&amp;g=214.0001004356&amp;u='+urlencode(this.href));" target="_blank" title="새해 벽두 카카오톡 먹통…2시간 수신·발신 장애">새해 벽두 <mark>카카오</mark>톡 먹통…2시간 수신·발신 장애</a>,
 <a class="news_tit" href="http://www.newsis.com/view/?id=NISX20200101_0000876715&amp;cID=13001&amp;pID=13000" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=55&amp;i=88000127_000000000000000009634453&amp;g=003.0009634453&amp;u='+urlencode(this.href));" target="_blank" title='카카오톡, 새해 첫날 오류…"내부 시스템 오류가 원인"'><mark>카카오</mark>톡, 새해 첫날 오류…"내부 시스템 오류가 원인"</a>,
 <a class="news_tit" href="https://view.asiae.co.kr/article/2020010111504955428" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=60&amp;i=88000385_000000000000000004598335&amp;g=277.0004598335&amp;u='+urlencode(this.href));" target="_blank" title="카카오톡, 새해 첫날부터 2시간 '먹통' 소동(종합)"><mark>카카오</mark

In [ ]:
# 기자 제목 추출
title[0]['title']

'새해 벽두 카카오톡 먹통…2시간 수신·발신 장애'

In [ ]:
title_list = [] # 기자제목 리스트
for i in title:
  title_list.append(i['title'])
title_list

['새해 벽두 카카오톡 먹통…2시간 수신·발신 장애',
 '카카오톡, 새해 첫날 오류…"내부 시스템 오류가 원인"',
 "카카오톡, 새해 첫날부터 2시간 '먹통' 소동(종합)",
 "카카오톡, 새해 시작부터 전송 오류…사용자들 '당황'",
 '‘지방 첫 대구 카카오택시 새해 새 출발’ 갈등 풀고 업무협약',
 'CU "올 간편결제 비중 10% 넘어"…삼성페이·카카오페이 순으로 많아',
 '서울보다 지역에서 잘달리는 카카오T블루',
 '카카오톡(카톡) 오류, 새해부터 답답한 서비스… 공식 입장은',
 '카카오톡 1일 0시부터 2시간여 먹통…“시스템 오류”',
 '카카오톡, 서버 오류로 인해 이용불가…누리꾼 "카톡 왜 안돼?"']

In [ ]:
# 기사 본문 추출을 위한 네이버 기사 링크 찾기
naver_article = soup.select('div.info_group')
naver_article

[<div class="info_group"> <a class="info press" href="http://imnews.imbc.com/" onclick="return goOtherCR(this, 'a=nws*h.prof&amp;r=50&amp;i=88000119_000000000000000001004356&amp;g=214.0001004356&amp;u='+urlencode(this.href));" target="_blank"><span class="thumb_box"><img alt="" class="thumb" data-lazysrc="https://search.pstatic.net/common/?src=https%3A%2F%2Fmimgnews.pstatic.net%2Fimage%2Fupload%2Foffice_logo%2F214%2F2017%2F01%2F05%2Flogo_214_18_20170105111205.png&amp;type=f54_54&amp;expire=24&amp;refresh=true" height="24" onerror="this.src='data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7';this.className='thumb bg_default_press'" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" width="24"/></span>MBC<i class="spnew ico_pick">언론사 선정</i></a><span class="info">2020.01.01.</span><a class="info" href="https://n.news.naver.com/mnews/article/214/0001004356?sid=102" onclick="return goOtherCR(this, 'a=nws*h.nav&amp;r=50&amp;i=880

In [ ]:
article_list = []
for i in naver_article:
  article_list.append(i)
article_list[0]

<div class="info_group"> <a class="info press" href="http://imnews.imbc.com/" onclick="return goOtherCR(this, 'a=nws*h.prof&amp;r=50&amp;i=88000119_000000000000000001004356&amp;g=214.0001004356&amp;u='+urlencode(this.href));" target="_blank"><span class="thumb_box"><img alt="" class="thumb" data-lazysrc="https://search.pstatic.net/common/?src=https%3A%2F%2Fmimgnews.pstatic.net%2Fimage%2Fupload%2Foffice_logo%2F214%2F2017%2F01%2F05%2Flogo_214_18_20170105111205.png&amp;type=f54_54&amp;expire=24&amp;refresh=true" height="24" onerror="this.src='data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7';this.className='thumb bg_default_press'" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" width="24"/></span>MBC<i class="spnew ico_pick">언론사 선정</i></a><span class="info">2020.01.01.</span><a class="info" href="https://n.news.naver.com/mnews/article/214/0001004356?sid=102" onclick="return goOtherCR(this, 'a=nws*h.nav&amp;r=50&amp;i=8800

In [ ]:
article_list[1]

<div class="info_group"> <a class="info press" href="http://www.newsis.com" onclick="return goOtherCR(this, 'a=nws*h.prof&amp;r=55&amp;i=88000127_000000000000000009634453&amp;g=003.0009634453&amp;u='+urlencode(this.href));" target="_blank"><span class="thumb_box"><img alt="" class="thumb" data-lazysrc="https://search.pstatic.net/common/?src=https%3A%2F%2Fmimgnews.pstatic.net%2Fimage%2Fupload%2Foffice_logo%2F003%2F2019%2F02%2F19%2Flogo_003_18_20190219111719.jpg&amp;type=f54_54&amp;expire=24&amp;refresh=true" height="24" onerror="this.src='data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7';this.className='thumb bg_default_press'" src="data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7" width="24"/></span>뉴시스<i class="spnew ico_pick">언론사 선정</i></a><span class="info">2020.01.01.</span><a class="info" href="https://n.news.naver.com/mnews/article/003/0009634453?sid=101" onclick="return goOtherCR(this, 'a=nws*h.nav&amp;r=55&amp;i=880001

In [ ]:
article_list[0].select('a')[0]['href'] # 언론사별 기사링크

'http://imnews.imbc.com/'

In [ ]:
article_list[0].select('a')[1]['href'] # 네이버뉴스 링크 => 링크가 없다면 이 코드는 작동하지 않음 => try except 방식으로 예외처리

'https://n.news.naver.com/mnews/article/214/0001004356?sid=102'

In [ ]:
article_list[1].select('a')[0]['href']

'http://www.newsis.com'

In [ ]:
article_list[1].select('a')[1]['href']

'https://n.news.naver.com/mnews/article/003/0009634453?sid=101'

In [ ]:
# 네이버 기사 링크 리스트로 정리
naver_article_url = []
for i in article_list:
  try:
    naver_article_url.append(i.select('a')[1]['href'])
  except:
    naver_article_url.append('url없음')
naver_article_url

['https://n.news.naver.com/mnews/article/214/0001004356?sid=102',
 'https://n.news.naver.com/mnews/article/003/0009634453?sid=101',
 'https://n.news.naver.com/mnews/article/277/0004598335?sid=105',
 'url없음',
 'https://n.news.naver.com/mnews/article/005/0001273999?sid=102',
 'https://n.news.naver.com/mnews/article/015/0004266650?sid=101',
 'https://n.news.naver.com/mnews/article/011/0003674404?sid=105',
 'url없음',
 'https://n.news.naver.com/mnews/article/056/0010778450?sid=105',
 'url없음']

In [ ]:
# 기사 날짜 추출
date = soup.select('div.info_group > span')
date_list = []
for i in date:
  date_list.append(i.text)
date_list

['2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 'A21면 1단',
 '2020.01.01.',
 '18면 2단',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.']

In [ ]:
# 날짜형식(xxxx.xx.xx.)이 나오게 수정
article_date = []
for i in date_list:
  p = re.compile('[0-9][0-9][0-9][0-9].[0-9][0-9].[0-9][0-9].')
  if p.match(i) != None:
    article_date.append(i)
article_date

['2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.',
 '2020.01.01.']

In [ ]:
# 기사 날짜 기사제목과 네이버 뉴스 url을 dataframe으로 정리
df_dic = {'기사날짜': article_date, '기사제목': title_list, '네이버 뉴스 url': naver_article_url}
df = pd.DataFrame(df_dic)
df

,기사날짜,기사제목,네이버 뉴스 url
0,2020.01.01.,새해 벽두 카카오톡 먹통…2시간 수신·발신 장애,https://n.news.naver.com/mnews/article/214/000...
1,2020.01.01.,"카카오톡, 새해 첫날 오류…""내부 시스템 오류가 원인""",https://n.news.naver.com/mnews/article/003/000...
2,2020.01.01.,"카카오톡, 새해 첫날부터 2시간 '먹통' 소동(종합)",https://n.news.naver.com/mnews/article/277/000...
3,2020.01.01.,"카카오톡, 새해 시작부터 전송 오류…사용자들 '당황'",url없음
4,2020.01.01.,‘지방 첫 대구 카카오택시 새해 새 출발’ 갈등 풀고 업무협약,https://n.news.naver.com/mnews/article/005/000...
5,2020.01.01.,"CU ""올 간편결제 비중 10% 넘어""…삼성페이·카카오페이 순으로 많아",https://n.news.naver.com/mnews/article/015/000...
6,2020.01.01.,서울보다 지역에서 잘달리는 카카오T블루,https://n.news.naver.com/mnews/article/011/000...
7,2020.01.01.,"카카오톡(카톡) 오류, 새해부터 답답한 서비스… 공식 입장은",url없음
8,2020.01.01.,카카오톡 1일 0시부터 2시간여 먹통…“시스템 오류”,https://n.news.naver.com/mnews/article/056/001...
9,2020.01.01.,"카카오톡, 서버 오류로 인해 이용불가…누리꾼 ""카톡 왜 안돼?""",url없음
